# Confidence Intervals for Various Quantities

### Section 7.2


## 95% Confidence Interval for Default Rate of Borrowers in Grade A


In [ ]:
import pandas as pd
import numpy as np
from main import preprocess_df
from collections import defaultdict as dd


df = pd.read_csv(
    "./datasets/lc_data_2007_to_2018.csv",
    low_memory=False,
    encoding="latin1",
    nrows=100000,  # only looking at 100k rows right now for performance
)
pd.set_option("display.max_columns", None)
cleaned_df = preprocess_df(df)